In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o") # temperature 0.7 (default)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from prompt import sundanese_overgeneration_prompt, sundanese_example_selector

chain = sundanese_overgeneration_prompt | model | StrOutputParser()

def generate_25_examples():
    return chain.batch([{"n": "5"}] * 5)

In [15]:
from tqdm.auto import tqdm

responses = []

for _ in tqdm(range(40)):
    res = generate_25_examples()
    responses.append(res)

  0%|          | 0/40 [00:00<?, ?it/s]

In [16]:
import re
def parse_generated_stories(res):
    # Split the input based on double new lines to separate each story
    res = re.sub(r'\d+\.\s*', '', res)

    stories = res.strip().split("\n\n")

    parsed_stories = []
    for story in stories:
        lines = story.split('\n')
        story_premise = lines[0].replace("Story Premise: ", "").strip()
        correct_ending = lines[1].replace("Correct Ending: ", "").strip()
        incorrect_ending = lines[2].replace("Incorrect Ending: ", "").strip()
        
        parsed_stories.append({
            "story_premise": story_premise,
            "correct_ending": correct_ending,
            "incorrect_ending": incorrect_ending
        })

    return parsed_stories

In [17]:
responses_formatted = []

[['Story Premise: Alit nuju meuli buku di toko buku. Anjeunna milih buku petualangan anu anyar. Alit resep maca buku dina waktu luangna. Anjeunna sering nganjuk kopi bari maca di taman.\nCorrect Ending: Buku petualangan anyar éta bakal ngajadikeun waktu soréna langkung seru\nIncorrect Ending: Alit hoyong ngajual bukuna ka batur\n\nStory Premise: Dina hiji peuting, Lilis ningali béntang-béntang di langit. Anjeunna ngimpi bisa nganjang ka luar angkasa. Anjeunna sering maca buku ngeunaan astronomi. Lilis hayang jadi astronom di mangsa depan.\nCorrect Ending: Anjeunna rajin diajar supados impianana janten kanyataan\nIncorrect Ending: Anjeunna mutuskeun pikeun jadi koki\n\nStory Premise: Rudi boga hiji anjing piaraan ngaran Bruno. Bruno sok ngiringan Rudi tiap indit ka taman. Rudi sok maén bal jeung Bruno di lapang. Salah sahiji poé, Bruno ngaleungit ti imah.\nCorrect Ending: Rudi milarian Bruno ka sabudeureun lingkungan sareng pamajikanana\nIncorrect Ending: Rudi mutuskeun pikeun henteu de